# Data Science Jobs Analyze


Group:
* Anchal
* Angela
* Larissa
* Fabio
* Felipe

This program extracts data from Indeed.ca website to analyze main caracteristics 
of Data Science Job opportunities in Toronto

In [1]:
#################### IMPORT LIBRARIES ####################

In [2]:
import datetime
import numpy as np
import pandas as pd
import re 
import requests
import datetime
from bs4 import BeautifulSoup
import time
import os

In [3]:
#################### SEARCH ARGUMENTS ####################

In [4]:
print('Start Time')
ts = time.time()
print (datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

input_job = "Data Scientist"
input_quote = False # add quotation marks("") to your input_job
input_city = "" # leave empty if input_city is not specified
input_state = "Canada"
sign = "+"
BASE_URL_indeed =  'https://ca.indeed.com/' #'http://www.indeed.com'


Start Time
2018-11-20 20:12:58


In [5]:
##### Function for Transform searching keywords #####
def transform(input,sign, quote = False):
    syntax = input.replace(" ", sign)
    if quote == True:
        syntax = ''.join(['"', syntax, '"'])
    return(syntax)

In [6]:
################# GENERATING BASE INDEED URL #################

In [7]:

if not input_city: # if (input_city is "")
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', input_state]
    url_indeed = ''.join(url_indeed_list)
else: # input_city is not ""
    url_indeed_list = [ BASE_URL_indeed, '/jobs?q=', transform(input_job, sign, input_quote),
                    '&l=', transform(input_city, sign), '%2C+', input_state]
    url_indeed = ''.join(url_indeed_list)

print(url_indeed)


https://ca.indeed.com//jobs?q=Data+Scientist&l=Canada


In [8]:
# get the HTML code from the URL
rawcode_indeed = requests.get(url_indeed)

# Choose "lxml" as parser
soup_indeed = BeautifulSoup(rawcode_indeed.text, "lxml")


In [9]:
# total number of results
num_total_indeed = soup_indeed.find(
                        id = 'searchCount').contents[0].split()[-2]

print("total results:")
print(soup_indeed.find(id = 'searchCount').contents[0])
print(soup_indeed.find(id = 'searchCount').contents[0].split()[-2])
num_total_indeed = re.sub("[^0-9]","", num_total_indeed) # remove non-numeric characters in the string
num_total_indeed = int(num_total_indeed)
#print(num_total_indeed)

# total number of pages
num_pages_indeed = int(np.ceil(num_total_indeed/10.0))
#print(num_pages_indeed)

# create an empty dataframe
df_base = pd.DataFrame()
# the date for today
now = datetime.datetime.now()
now_str = now.strftime("%m/%d/%Y")
now_str_name=now.strftime('%m%d%Y')

total results:

        Page 1 of 1,052 jobs
1,052


In [10]:
############ function to clean text before scraping ########### 

In [11]:
def text_cleaner(dirty_text):

    text = dirty_text

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    text = ''.join(chunk for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line
    
    # Now clean out all of the unicode junk (this line works great!!!)
    
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception
   
    #print(text)
    text = text.decode('utf-8')
    text = re.sub("[^a-zA-Z+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                             # Also include + for C++
    text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text) # Fix spacing issue from merged words
    
    text = text.lower().split()  # Go to lower case and split them apart
    
    
   # stop_words = set(stopwords.words("english")) # Filter out any stop words
   # text = [w for w in text if not w in stop_words]
    
    text = list(set(text)) # Last, just get the set of these. Ignore counts (we are just looking at whether a term existed
                           # or not on the website)
    
    return text

######### CREATING DATA FRAME WITH ALL JOB OPPORTUNITIES URLS #####

In [12]:
######## functions ###########
def Salary(soup):
    try:
        salary = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item'})
        return  salary.contents
    except:
        return 'Not Informed'
    
def Job_Type(soup):
    try:
        rawcode = requests.get(url)
        soup = BeautifulSoup(rawcode.text, "html.parser")
        job_type = soup.find( attrs = {'class' : 'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs'})
        return  job_type.contents
    except:
        return 'Not Informed'


In [13]:
########################################
##### Loop for all the total pages #####
########################################

#num_pages_indeed = 1 
df_base = pd.DataFrame()
count = 0

for i in range(1, num_pages_indeed+1):
    # generate the URL
    url = ''.join([url_indeed, '&start=', str(i*10)])
    #print(url)

    # get  LXML
    rawcode = requests.get(url)
    soup_lxml = BeautifulSoup(rawcode.text, "lxml")
    
    
    #Extracting the Salary


    # pick out all the "div" with "class="job-row"
    divs = soup_lxml.findAll("div")
    job_divs = [jp for jp in divs if not jp.get('class') is None
                    and 'row' in jp.get('class')]

    
    for job in job_divs:
        try:
            
            ix = job.index
            # job id
            id = job.get('data-jk', None)
            #print(id)
            
            # job link related to job id
            
            link = BASE_URL_indeed + '/viewjob'+ '?jk=' + id
            #link = 'https://ca.indeed.com/Data-Scientist-jobs-in-Canada?vjk=' + id
            #print(link)
            
            # job title
            title = job.find('a', attrs={'data-tn-element': 'jobTitle'}).attrs['title']
            #print(title)
            
            #sja = 'sja'+ str(count)
            #href = BASE_URL_indeed + job.find('a', attrs={'id': sja}).attrs['href']
            
            
            #print(href)
            #count = count + 1
            # job company
            company = job.find('span', {'class': 'company'}).text.strip()
            #print(company)
            
            # job location
            location = job.find('span', {'class': 'location'}).text.strip()
            #print(location)
            #city, province
            city,province = location.split(',')
            
            ### html.parser - Felipe
            r = requests.get(link)
            soup_html = BeautifulSoup(r.text, "html.parser")
            #print(soup_html)
            
            # salary
            salary = Salary(soup_html)
            #print('salary:' + salary)
            
            # job type
            jobtype = Job_Type(soup_html)

            
            #Extracting the Job Title
            position = soup_html.find(attrs = {'class' : 'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'})
            position = position.contents
            
           
            # job Requirements
            try:   
                #page = requests.get(link)
                #soup = BeautifulSoup(page.text, 'html.parser')
                text  = soup_html.get_text()
                jobrequirements = text
                #jobrequirements = text_cleaner(text)
                
            except:

                jobrequirements = 'Forbidden'

            
        except:
            continue
        count = count + 1
        df_base = df_base.append({'Job_ID': id,
                                  'JobTitle': title,
                                  'JobType':jobtype,
                                  'CompanyName': company,
                                  'Salary' : salary ,
                                  'Position':position,
                                  'City': city,
                                  'Province':province,
                                  'JobRequirements':jobrequirements,
                                  'Date': now_str,
                                  'From':"Indeed",
                                  'JobLink':link},ignore_index=True)

        print(str(count) +  ' jobs added to the data frame' )



1 jobs added to the data frame
2 jobs added to the data frame
3 jobs added to the data frame
4 jobs added to the data frame
5 jobs added to the data frame
6 jobs added to the data frame
7 jobs added to the data frame
8 jobs added to the data frame
9 jobs added to the data frame
10 jobs added to the data frame
11 jobs added to the data frame
12 jobs added to the data frame
13 jobs added to the data frame
14 jobs added to the data frame
15 jobs added to the data frame
16 jobs added to the data frame
17 jobs added to the data frame
18 jobs added to the data frame
19 jobs added to the data frame
20 jobs added to the data frame
21 jobs added to the data frame
22 jobs added to the data frame
23 jobs added to the data frame
24 jobs added to the data frame
25 jobs added to the data frame
26 jobs added to the data frame
27 jobs added to the data frame
28 jobs added to the data frame
29 jobs added to the data frame
30 jobs added to the data frame
31 jobs added to the data frame
32 jobs added to 

253 jobs added to the data frame
254 jobs added to the data frame
255 jobs added to the data frame
256 jobs added to the data frame
257 jobs added to the data frame
258 jobs added to the data frame
259 jobs added to the data frame
260 jobs added to the data frame
261 jobs added to the data frame
262 jobs added to the data frame
263 jobs added to the data frame
264 jobs added to the data frame
265 jobs added to the data frame
266 jobs added to the data frame
267 jobs added to the data frame
268 jobs added to the data frame
269 jobs added to the data frame
270 jobs added to the data frame
271 jobs added to the data frame
272 jobs added to the data frame
273 jobs added to the data frame
274 jobs added to the data frame
275 jobs added to the data frame
276 jobs added to the data frame
277 jobs added to the data frame
278 jobs added to the data frame
279 jobs added to the data frame
280 jobs added to the data frame
281 jobs added to the data frame
282 jobs added to the data frame
283 jobs a

502 jobs added to the data frame
503 jobs added to the data frame
504 jobs added to the data frame
505 jobs added to the data frame
506 jobs added to the data frame
507 jobs added to the data frame
508 jobs added to the data frame
509 jobs added to the data frame
510 jobs added to the data frame
511 jobs added to the data frame
512 jobs added to the data frame
513 jobs added to the data frame
514 jobs added to the data frame
515 jobs added to the data frame
516 jobs added to the data frame
517 jobs added to the data frame
518 jobs added to the data frame
519 jobs added to the data frame
520 jobs added to the data frame
521 jobs added to the data frame
522 jobs added to the data frame
523 jobs added to the data frame
524 jobs added to the data frame
525 jobs added to the data frame
526 jobs added to the data frame
527 jobs added to the data frame
528 jobs added to the data frame
529 jobs added to the data frame
530 jobs added to the data frame
531 jobs added to the data frame
532 jobs a

751 jobs added to the data frame
752 jobs added to the data frame
753 jobs added to the data frame
754 jobs added to the data frame
755 jobs added to the data frame
756 jobs added to the data frame
757 jobs added to the data frame
758 jobs added to the data frame
759 jobs added to the data frame
760 jobs added to the data frame
761 jobs added to the data frame
762 jobs added to the data frame
763 jobs added to the data frame
764 jobs added to the data frame
765 jobs added to the data frame
766 jobs added to the data frame
767 jobs added to the data frame
768 jobs added to the data frame
769 jobs added to the data frame
770 jobs added to the data frame
771 jobs added to the data frame
772 jobs added to the data frame
773 jobs added to the data frame
774 jobs added to the data frame
775 jobs added to the data frame
776 jobs added to the data frame
777 jobs added to the data frame
778 jobs added to the data frame
779 jobs added to the data frame
780 jobs added to the data frame
781 jobs a

1000 jobs added to the data frame
1001 jobs added to the data frame
1002 jobs added to the data frame
1003 jobs added to the data frame
1004 jobs added to the data frame
1005 jobs added to the data frame
1006 jobs added to the data frame
1007 jobs added to the data frame
1008 jobs added to the data frame
1009 jobs added to the data frame
1010 jobs added to the data frame
1011 jobs added to the data frame
1012 jobs added to the data frame
1013 jobs added to the data frame
1014 jobs added to the data frame
1015 jobs added to the data frame
1016 jobs added to the data frame
1017 jobs added to the data frame
1018 jobs added to the data frame
1019 jobs added to the data frame
1020 jobs added to the data frame
1021 jobs added to the data frame
1022 jobs added to the data frame
1023 jobs added to the data frame
1024 jobs added to the data frame
1025 jobs added to the data frame
1026 jobs added to the data frame
1027 jobs added to the data frame
1028 jobs added to the data frame
1029 jobs adde

In [14]:
try:
    os.remove('out.csv')
except: 
    print('file removed')

file removed


In [15]:
df_base.to_csv('out.csv')

In [16]:
############# READING DATAFRAME AND SCRAPING JOB DESCRIPTION ############

In [17]:
#df_base.JobRequirements.values

In [18]:
skill_source = pd.read_excel('ds.xlsx')
kw = pd.read_excel('ds_kw.xlsx')
skill_source

,Skill_ID,Skill,JobTitle,Category
0,-1,Not Identified,Data Scientist,Not Identified
1,1,Python,Data Scientist,Hard
2,2,SQL,Data Scientist,Hard
3,3,BigData,Data Scientist,Hard
4,4,NumPy,Data Scientist,Hard
5,5,Spring,Java Developer,Hard
6,6,Mathematcs,Data Scientist,Education
7,7,SQL,Java Developer,Hard
8,8,Statistics,Data Scientist,Major
9,9,Machine Learning,Data Scientist,Hard


In [19]:

# Functions
def find_skill(skill, jobReq ):
    if skill in jobReq:
        return True
    else:
        return False

In [20]:
df_skill_aux = pd.DataFrame(columns=['Job_ID', 'KeyWord', 'Skill_ID'])


In [21]:
df_skill_aux

,Job_ID,KeyWord,Skill_ID


In [22]:
z = 0
for index, row in df_base.iterrows():
    k = kw.loc[kw.JobTitle == row["JobTitle"], ['KeyWord', 'Skill_ID']]
    jobReq = row["JobRequirements"]
    job_ID = row["Job_ID"]

    for idx, skills in k.iterrows():
        skill = ' ' + skills["KeyWord"]
        skill_ID = skills["Skill_ID"]
        func = find_skill(skill, jobReq)
        if func:
            # print(job_ID, skill, skill_ID)
            df_skill_aux.loc[z] = [job_ID, skill, skill_ID]
            z += 1




In [23]:
df_skill_aux.head()

,Job_ID,KeyWord,Skill_ID
0,7796d7431bb700e7,Python,1
1,7796d7431bb700e7,ML,9
2,7796d7431bb700e7,R,11
3,7796d7431bb700e7,Master,13
4,7796d7431bb700e7,Masters,13


In [24]:
# Creating df_tmp
df_tmp = pd.DataFrame()


In [25]:
df_tmp = df_base.merge(df_skill_aux, how='left', left_on='Job_ID', right_on='Job_ID')
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Python,1
1,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,ML,9
2,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,R,11
3,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Master,13
4,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Masters,13


In [26]:
############# Treating null values #################
values = {'Skill_ID': '-1', 'KeyWord': "Not found"}
df_tmp = df_tmp.fillna(value=values)

In [27]:
df_tmp['Skill_ID'] = df_tmp['Skill_ID'].astype(int)

In [28]:
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobRequirements,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Python,1
1,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,ML,9
2,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,R,11
3,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Master,13
4,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,"\n\nData Scientist - Toronto, ON - Indeed.com\...",Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Masters,13


In [29]:
skill_source.head()

,Skill_ID,Skill,JobTitle,Category
0,-1,Not Identified,Data Scientist,Not Identified
1,1,Python,Data Scientist,Hard
2,2,SQL,Data Scientist,Hard
3,3,BigData,Data Scientist,Hard
4,4,NumPy,Data Scientist,Hard


In [30]:
####### Use this in case we want to filter jobs that does not match with keywords

#filters Skill_ID not identified
#df_filtered = df_tmp.query('Skill_ID>0')
#df_filtered.head()



In [31]:
#drop description colum to avoid a big data fram
df_tmp = df_tmp.drop(columns=['JobRequirements'])
df_tmp.head()

,City,CompanyName,Date,From,JobLink,JobTitle,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID
0,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Python,1
1,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,ML,9
2,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,R,11
3,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Master,13
4,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Masters,13


In [32]:

df_jobs = df_tmp.merge(skill_source, left_on='Skill_ID', right_on='Skill_ID', how='left')
df_jobs.head()

,City,CompanyName,Date,From,JobLink,JobTitle_x,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Python,1,Python,Data Scientist,Hard
1,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,ML,9,Machine Learning,Data Scientist,Hard
2,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,R,11,R,Data Scientist,Hard
3,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Master,13,Master,Data Scientist,Education
4,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Masters,13,Master,Data Scientist,Education


In [33]:
df_jobs = df_jobs.drop_duplicates(subset={'Job_ID','Skill_ID'}, keep='first', inplace=False)

In [34]:
df_jobs.head()

,City,CompanyName,Date,From,JobLink,JobTitle_x,JobType,Job_ID,Position,Province,Salary,KeyWord,Skill_ID,Skill,JobTitle_y,Category
0,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Python,1,Python,Data Scientist,Hard
1,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,ML,9,Machine Learning,Data Scientist,Hard
2,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,R,11,R,Data Scientist,Hard
3,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Master,13,Master,Data Scientist,Education
5,Toronto,Maple Leaf Sports & Entertainment,11/20/2018,Indeed,https://ca.indeed.com//viewjob?jk=7796d7431bb7...,Data Scientist,Not Informed,7796d7431bb700e7,[Data Scientist],ON,Not Informed,Excel,15,Excel,Data Scientist,Hard


In [35]:
os.remove('out_jobs_opp.csv')
df_final=df_jobs.to_csv('out_jobs_opp.csv')

In [36]:
ts = time.time()
print('End Time')
print (datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

End Time
2018-11-20 20:33:31
